In [1]:
import os
import sys
import numpy as np
from skimage import io
import itk
from itkwidgets import compare, checkerboard, view

In [ ]:
#target is the Allen annotation volume
TARGET_LABEL_IMAGE_FILEPATH = '../data/allen_mouse_ccf/annotation/ccf_2017/annotation_25.nii.gz'
INPUT_RESULTS_PATH = '../data/631680_label_image_results/631680_Ex_647_Em_690'
INPUT_LABEL_IMAGE_FILEPATH = f'{INPUT_RESULTS_PATH}/631680_Caudoputamen_transformed.nii.gz'
INPUT_POINT_SET_FILEPATH = f'{INPUT_RESULTS_PATH}/outputpoints.txt'
target_label_image = itk.imread(TARGET_LABEL_IMAGE_FILEPATH)
target_label_image_f = itk.cast_image_filter(target_label_image, ttype=[type(target_label_image), 
                                                                        itk.Image[itk.F,3]])
target_binary_image = itk.binary_threshold_image_filter(
    target_label_image_f,
    lower_threshold=TARGET_LABEL_VALUE,
    upper_threshold=TARGET_LABEL_VALUE,
    inside_value=1,
    outside_value=0,
    ttype=[type(target_label_image_f), itk.Image[itk.UC,target_label_image_f.GetImageDimension()]]
)

print(f'Binary labels: {np.unique(target_binary_image)}')

In [ ]:
transformed_source_label_image = itk.imread(INPUT_LABEL_IMAGE_FILEPATH)
dice_score = compute_dice_coefficient(transformed_source_label_image, target_binary_image)

print(f'Evaluated dice value: {dice_score}')


In [2]:
def compute_dice_coefficient(source_image: itk.Image, target_image: itk.Image) -> float:
    """Compute the dice coefficient to compare volume overlap between two label regions"""
    dice_filter = itk.LabelOverlapMeasuresImageFilter[type(source_image)].New()
    dice_filter.SetInput(source_image)
    dice_filter.SetTargetImage(target_image)
    dice_filter.Update()
    return dice_filter.GetDiceCoefficient()

In [14]:
DATA = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/CH1/thumbnail_cleaned'
file100path = os.path.join(DATA, '225.tif')
file101path = os.path.join(DATA, '226.tif')

In [15]:
file100img = io.imread(file100path)
file100img = itk.GetImageFromArray(np.ascontiguousarray(file100img))
file101img = io.imread(file101path)
file101img = itk.GetImageFromArray(np.ascontiguousarray(file101img))

In [5]:
binary100 = itk.binary_threshold_image_filter(
    file100img,
    lower_threshold=0,
    upper_threshold=15,
    inside_value=1,
    outside_value=0,
    ttype=[type(file100img), itk.Image[itk.UC, file100img.GetImageDimension()]]
)
TARGET_LABEL_VALUE = 999
binary101 = itk.binary_threshold_image_filter(
    file101img,
    lower_threshold=TARGET_LABEL_VALUE,
    upper_threshold=TARGET_LABEL_VALUE,
    inside_value=1,
    outside_value=0,
    ttype=[type(file101img), itk.Image[itk.UC, file101img.GetImageDimension()]]
)

In [6]:
view(binary100)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC2; pr…

In [7]:
view(binary101)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC2; pr…

In [8]:
# cleaned = 0.9412216469382709
# aligned = 0.992935077024429
dice_score = compute_dice_coefficient(binary100, binary101)
print(f'Evaluated dice value: {dice_score}')

Evaluated dice value: 0.992935077024429


In [18]:
compare(file100img, file101img, link_cmap=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=True, description='cmap'), Checkbox(va…